In [1]:
import json

import torch
from torch import nn
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data.dataloader import DataLoader
from torchvision import transforms
from torchvision.models import ResNet
from custom_image_folder import CustomImageFolder
import chromadb
from custom_cnn.networks.model_skipped_18_layer_for_embedding import Skipped18LayerForEmbbeding
from custom_cnn import utility_functions

## Prepare embedding model

In [2]:
#model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18')
#torch.save(model.state_dict, "/home/erik/Desktop/WiSe2023_24/xAI-Proj-M/xAI-Proj-M/embeddings/pretrained_models/resnet18_pretrained")
model = Skipped18LayerForEmbbeding()
model.network.load_state_dict(
    torch.load("/home/erik/Desktop/WiSe2023_24/xAI-Proj-M/xAI-Proj-M/custom_cnn/savedmodels/TODOOOOOOtrain this model", map_location=torch.device(utility_functions.get_default_device()))
)

Set up Modular Skip
in: 16, out: 16, stride: (1, 1)
No skip conv needed, use identity
Set up Modular Skip
in: 16, out: 32, stride: (2, 2)
Create skip conv
Set up Modular Skip
in: 32, out: 32, stride: (1, 1)
No skip conv needed, use identity
Set up Modular Skip
in: 32, out: 64, stride: (2, 2)
Create skip conv
Set up Modular Skip
in: 64, out: 64, stride: (1, 1)
No skip conv needed, use identity
Set up Modular Skip
in: 64, out: 128, stride: (2, 2)
Create skip conv
Set up Modular Skip
in: 128, out: 128, stride: (1, 1)
No skip conv needed, use identity
Set up Modular Skip
in: 128, out: 128, stride: (1, 1)
No skip conv needed, use identity


RuntimeError: Error(s) in loading state_dict for Sequential:
	Unexpected key(s) in state_dict: "29.weight", "29.bias". 

In [15]:
print(F"type of model {type(model)}")
print(f"{model.fc}")

type of model <class 'custom_cnn.networks.model_skipped_18_layer.Skipped18Layer'>


AttributeError: 'Skipped18Layer' object has no attribute 'fc'

In [3]:
# Replace last FC layer by identity mapping so that the model outputs vectors of dim 512
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

model.fc = Identity()

## Prepare vector store

In [5]:
chroma_client = chromadb.PersistentClient(path="/home/erik/Desktop/WiSe2023_24/xAI-Proj-M/xAI-Proj-M/embeddings/embedding_data/resnet18_embedding_store")

collection = chroma_client.create_collection(name="pretrained_resnet18_embeddings")
#collection = chroma_client.get_collection(name="my_collection")
print(f"Samples in db: {len(collection.get()['embeddings'])}")

TypeError: object of type 'NoneType' has no len()

In [6]:
def get_embeddings_and_store(embedding_model, data_loader, chroma_collection, dataset_type):
    count = 0
    for batch in data_loader:
        print(count)
        batch_paths_and_images, batch_labels = batch
        batch_labels = batch_labels.tolist()
        batch_paths = list(batch_paths_and_images[0])
        batch_images = batch_paths_and_images[1]
        
        out = embedding_model(batch_images)
        
        chroma_collection.add(
            embeddings = out.tolist(),
            metadatas = [{'path':x[0], 'label':x[1], 'dataset_type':dataset_type} for x in zip(batch_paths, batch_labels)],
            ids= [dataset_type + str(x) for x in list(range( count * 256, count * 256 + len(batch_labels)))]
        )

        count += 1


## Get embeddings of train dataqset

In [8]:
# Use custom Dataset that returns image path in get item
train_dataset = CustomImageFolder(
    root="/home/erik/Desktop/WiSe2023_24/xAI-Proj-M/xAI-Proj-M/custom_cnn/data/cifar10/train",
    transform=ToTensor()
)
# Create DataLoader
train_data_loader = DataLoader(
    dataset=train_dataset,
    batch_size=256,
    shuffle=False,
    num_workers=4,
    pin_memory=False  # should be True if cuda is available
)
# feed through embedding model
model.eval()
get_embeddings_and_store(model, train_data_loader, collection, dataset_type="train")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195


In [9]:
print(f"Samples in db: {len(collection.get()['ids'])}")
print(collection.get()['ids'][0])
print(collection.get()['ids'][-1])

Samples in db: 50000
train0
train49999


## Get embeddings of test dataset

In [10]:
# Use custom Dataset that returns image path in get item
test_dataset = CustomImageFolder(
    root="/home/erik/Desktop/WiSe2023_24/xAI-Proj-M/xAI-Proj-M/custom_cnn/data/cifar10/test",
    transform=ToTensor()
)
# Create DataLoader
test_data_loader = DataLoader(
    dataset=test_dataset,
    batch_size=256,
    shuffle=False,
    num_workers=4,
    pin_memory=False  # should be True if cuda is available
)
# feed through embedding model
get_embeddings_and_store(model, test_data_loader, chroma_collection=collection, dataset_type="test")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [11]:
print(f"Samples in db: {len(collection.get()['ids'])}")
print(collection.get()['ids'][50000])
print(collection.get()['ids'][-1])

Samples in db: 60000
test0
test9999


In [12]:
collection.get(include=['embeddings', 'documents', 'metadatas'])['embeddings'][0]

[0.0,
 0.15496140718460083,
 0.6533552408218384,
 0.10460175573825836,
 0.23001089692115784,
 0.015446186996996403,
 0.3059708774089813,
 0.03321392089128494,
 0.039559271186590195,
 0.12876932322978973,
 0.03549429029226303,
 0.3491635024547577,
 0.06376586109399796,
 0.05076976120471954,
 0.06907577812671661,
 0.0,
 0.2906697988510132,
 0.04119785875082016,
 0.0,
 0.0,
 0.0,
 0.3092459738254547,
 0.1780966818332672,
 0.18186092376708984,
 0.3881151080131531,
 0.777580976486206,
 0.1032431498169899,
 0.8182851672172546,
 0.046224966645240784,
 0.3971695899963379,
 0.03695961460471153,
 0.12913483381271362,
 0.2958517074584961,
 0.05004793405532837,
 0.05118536576628685,
 0.04675797373056412,
 0.0,
 0.9347991943359375,
 0.0,
 0.024805961176753044,
 0.5680094957351685,
 0.0858125239610672,
 0.0,
 0.22595395147800446,
 0.0,
 0.000975943636149168,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.06971263140439987,
 0.044268764555454254,
 0.0,
 0.0,
 1.156774640083313,
 0.018786532804369926,
 0.1977242827